In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
url = 'https://meteostat.net/en/place/vn/thu-uc?s=48900&t=2024-05-15/2024-05-26'

In [4]:
driver.get(url)

In [5]:
button_table_xpath = '//*[@id="app"]/div/main/div/div/div/div[1]/div[1]/div[1]/button[2]'
button_table = driver.find_element(By.XPATH, button_table_xpath)
driver.execute_script("arguments[0].click();", button_table)

In [6]:
table_xpath = '//*[@id="details-modal"]/div/div/div[2]/div[1]'
table = driver.find_element(By.XPATH, table_xpath)

In [7]:
thead_xpath = '//*[@id="details-modal"]/div/div/div[2]/div[1]/table/thead'
thead = table.find_element(By.XPATH, thead_xpath)

thead_tr_xpath = '//*[@id="details-modal"]/div/div/div[2]/div[1]/table/thead/tr'
thead_tr = table.find_element(By.XPATH, thead_tr_xpath)
thead_tr_th_xpath = '//*[@id="details-modal"]/div/div/div[2]/div[1]/table/thead/tr/th'
thead_tr_th_list = thead_tr.find_elements(By.XPATH, thead_tr_th_xpath)

th_list = [th.text for th in thead_tr_th_list]

In [8]:
button_showmore_xpath = '//*[@id="details-modal"]/div/div/div[2]/div[2]/button'
button_showmore = driver.find_element(By.XPATH, button_showmore_xpath)

In [10]:
while True:
    if not button_showmore.get_attribute('disabled'):
        driver.execute_script("arguments[0].click();", button_showmore)
    else:
        break

In [11]:
tbody_xpath = '//*[@id="details-modal"]/div/div/div[2]/div[1]/table/tbody'
tbody = driver.find_element(By.XPATH, tbody_xpath)
tbody_tr_xpath = '//*[@id="details-modal"]/div/div/div[2]/div[1]/table/tbody/tr'
tbody_tr_list = tbody.find_elements(By.XPATH, tbody_tr_xpath)

len(tbody_tr_list)

288

In [35]:
td = list()
for tr in tbody_tr_list:
    # Extract row's ./th
    tr_th = tr.find_elements(By.XPATH, './th')
    tr_th = [i.text for i in tr_th]
    
    # Extract row's ./td
    tr_td_list = tr.find_elements(By.XPATH, './td')
    
    # Extract coco
    tr_td_coco = tr_td_list[0]
    tr_td_coco_class = tr_td_coco.find_element(By.XPATH, './i').get_attribute('class')
    tr_td_coco = tr_td_coco_class.split('-')[-1]
    
    # Extract wind direction from element's class
    tr_wdir = tr_td_list[6]
    tr_wdir_class = tr_wdir.find_element(By.XPATH, './i').get_attribute('class')
    tr_wdir = tr_wdir_class.split('-')[-2]
    
    # Assemble row's data
    tr_td_list = tr_th + [td.text for td in tr_td_list]
    tr_td_list[8] = tr_wdir
    tr_td_list[2] = tr_td_coco
    td.append(tr_td_list)

In [37]:
for i in td:
    print(i)

['2024-05-15', '00', 'cloudy', '31 °C', '26 °C', '—', '0 mm', '—', '150', '9 km/h', '—', '1009 hPa', '75 %']
['2024-05-15', '01', 'cloudy', '30 °C', '26 °C', '—', '0 mm', '—', '150', '7 km/h', '—', '1008 hPa', '79 %']
['2024-05-15', '02', 'cloudy', '30 °C', '27 °C', '—', '0 mm', '—', '140', '13 km/h', '—', '1008 hPa', '84 %']
['2024-05-15', '03', 'cloudy', '30 °C', '27 °C', '—', '0 mm', '—', '130', '9 km/h', '—', '1008 hPa', '84 %']
['2024-05-15', '04', 'cloudy', '30 °C', '27 °C', '—', '0 mm', '—', '120', '4 km/h', '—', '1009 hPa', '84 %']
['2024-05-15', '05', 'cloudy', '30 °C', '27 °C', '—', '0 mm', '—', '130', '11 km/h', '—', '1009 hPa', '84 %']
['2024-05-15', '06', 'cloudy', '31 °C', '26.9 °C', '—', '0 mm', '—', '120', '13 km/h', '—', '1009 hPa', '79 %']
['2024-05-15', '07', 'cloudy', '32 °C', '26.1 °C', '—', '0 mm', '—', '100', '15 km/h', '—', '1011 hPa', '71 %']
['2024-05-15', '08', 'cloudy', '33 °C', '25 °C', '—', '0 mm', '—', '110', '19 km/h', '—', '1011 hPa', '63 %']
['2024-05-

In [39]:
meteostat_df = pd.DataFrame(td, columns=th_list)

In [40]:
meteostat_df

,Date,H,COCO,TEMP,DWPT,TSUN,PRCP,SNOW,WDIR,WSPD,WPGT,PRES,RHUM
0,2024-05-15,00,cloudy,31 °C,26 °C,—,0 mm,—,150,9 km/h,—,1009 hPa,75 %
1,2024-05-15,01,cloudy,30 °C,26 °C,—,0 mm,—,150,7 km/h,—,1008 hPa,79 %
2,2024-05-15,02,cloudy,30 °C,27 °C,—,0 mm,—,140,13 km/h,—,1008 hPa,84 %
3,2024-05-15,03,cloudy,30 °C,27 °C,—,0 mm,—,130,9 km/h,—,1008 hPa,84 %
4,2024-05-15,04,cloudy,30 °C,27 °C,—,0 mm,—,120,4 km/h,—,1009 hPa,84 %
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2024-05-26,19,cloudy,31 °C,26 °C,—,0.5 mm,—,200,13 km/h,—,1007 hPa,75 %
284,2024-05-26,20,cloudy,31 °C,26 °C,—,0 mm,—,190,11 km/h,—,1008 hPa,75 %
285,2024-05-26,21,cloudy,30 °C,26 °C,—,0 mm,—,230,11 km/h,—,1009 hPa,79 %
286,2024-05-26,22,cloudy,30 °C,26 °C,—,0 mm,—,230,7 km/h,—,1009 hPa,79 %
